# Training and Testing

## Loading Data

In [1]:
## Import package and functions
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.utils.data import DataLoader
from CNN import *
from data_loader import *
from CRNN import *
from train_test import *
import time

%load_ext autoreload
%autoreload 2

In [4]:
#Load and preprocess data
path = "project_data/"
X_train_valid, y_train_valid, X_test, y_test = data_loader(path,True)
X_train_valid, y_train_valid, X_test, y_test = data_process(X_train_valid, 
        X_test, y_train_valid, y_test, verbose = True)
x_shape,train_dataset = Dataset_torch(X_train_valid,y_train_valid,verbose=True)
x_test_shape,test_dataset = Dataset_torch(X_test,y_test,verbose=True)

The X train & valid data size is (2115, 22, 1000)
The X test data size is (443, 22, 1000)
Shape of X train after trimming is: (2115, 22, 500)
Shape of X train after maxpooling: (2115, 22, 250)
Shape of X test after maxpooling: (443, 22, 250)
Shape of X train after averaging: (4230, 22, 250)
Shape of X test after averaging: (886, 22, 250)
Shape of X train after subsampling and concatenating: (8460, 22, 250)
Shape of Y train after subsampling and concatenating: (8460,)
Shape of X test after subsampling and concatenating: (1772, 22, 250)
Shape of Y train after subsampling and concatenating: (1772,)
Shape of x set after adding width info: (8460, 22, 250, 1)
Shape of x set after adding width info: (1772, 22, 250, 1)


In [5]:
X_train_valid2, y_train_valid2, X_test2, y_test2 = data_loader(path,True)
X_train_valid2, y_train_valid2, X_test2, y_test2 = crop_data(X_train_valid2, 
        X_test2, y_train_valid2, y_test2, data_crop = 0.5, verbose = True)
x_shape2,train_dataset2 = Dataset_torch(X_train_valid2,y_train_valid2,verbose=True)
x_test_shape2,test_dataset2 = Dataset_torch(X_test2,y_test2,verbose=True)

The X train & valid data size is (2115, 22, 1000)
The X test data size is (443, 22, 1000)
Shape of X train after trimming is: (2115, 22, 500)
Shape of x set after adding width info: (2115, 22, 500, 1)
Shape of x set after adding width info: (443, 22, 500, 1)


## Hyperparameter

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
num_epoch = 100
lr = 3e-4
dropout = 0.5
k_folds = 5
bidirectional = False
lr_decay = True
verbose = True
subsampler = True
print_every = 25
criterion = nn.CrossEntropyLoss()


## 1.CNN

In [3]:
model_CNN = train(x_shape, 
                  train_dataset, 
                  'CNN', 
                  criterion, 
                  k_folds = k_folds,
                  num_epochs = num_epoch, 
                  dropout = dropout,
                  print_every = print_every,
                  subsampler = True, 
                  lr_decay=True, 
                  verbose = True, 
                  bidirection = False)

Using cross validation with fold modes KFold(n_splits=5, random_state=None, shuffle=True)
===============FOLD 0===============
Model Initialized


C:\Users\Rong\anaconda3\lib\site-packages\torch\nn\modules\conv.py:442: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  ..\aten\src\ATen\native\Convolution.cpp:647.)
  return F.conv2d(input, weight, bias, self.stride,


Loss afer 25 epoch: 0.677, with the accuracy 73.750
The validation accuracy at epoch 25 is: 89.06
Loss afer 50 epoch: 0.463, with the accuracy 82.438
The validation accuracy at epoch 50 is: 93.75
Loss afer 75 epoch: 0.353, with the accuracy 86.984
The validation accuracy at epoch 75 is: 97.66
Loss afer 100 epoch: 0.308, with the accuracy 88.859
The validation accuracy at epoch 100 is: 99.22
Deleted mode
===============FOLD 1===============
Model Initialized
Loss afer 25 epoch: 0.668, with the accuracy 74.141
The validation accuracy at epoch 25 is: 85.16
Loss afer 50 epoch: 0.462, with the accuracy 82.266
The validation accuracy at epoch 50 is: 89.84
Loss afer 75 epoch: 0.341, with the accuracy 87.172
The validation accuracy at epoch 75 is: 99.22
Loss afer 100 epoch: 0.287, with the accuracy 89.172
The validation accuracy at epoch 100 is: 100.00
Deleted mode
===============FOLD 2===============
Model Initialized
Loss afer 25 epoch: 0.674, with the accuracy 73.656
The validation accuracy

In [4]:
acc_last_model= test(test_dataset, model_CNN)

The test accuracy is: 71.05 %


In [4]:
#Suppress cross validation
start_time = time.time()
num_epoch = 100
dropout = 0.5
cross_valid = False
saving_file = False
model_CNN = train(x_shape, 
                  train_dataset, 
                  'CNN', 
                  criterion, 
                  k_folds = k_folds,
                  num_epochs = num_epoch, 
                  dropout = dropout,
                  print_every = print_every,
                  subsampler = True, 
                  lr_decay=True, 
                  verbose = True,
                  cross_valid = False,
                  bidirection = False,
                  if_save = True)
acc_last_model= test(test_dataset, model_CNN)
print("70 Epochs for CNN need %s seconds ---" % (time.time() - start_time))

Cross validation is not implemented
===============FOLD 0===============
Model Initialized
Loss afer 25 epoch: 0.664, with the accuracy 74.375
The validation accuracy at epoch 25 is: 85.16
Loss afer 50 epoch: 0.453, with the accuracy 82.594
The validation accuracy at epoch 50 is: 92.97
Loss afer 75 epoch: 0.361, with the accuracy 86.234
The validation accuracy at epoch 75 is: 98.44
Loss afer 100 epoch: 0.285, with the accuracy 89.297
The validation accuracy at epoch 100 is: 97.66
All 5 Folds Completed
Fold 0: 98.06466590033637 %
Average is:  98.06466590033637 %
Saving training results...
The test accuracy is: 73.19 %
70 Epochs for CNN need 81.04152274131775 seconds ---


# 2.LSTM

In [7]:
start_time = time.time()
num_epoch = 80
dropout = 0.5
cross_valid = False
model_LSTM = train(x_shape, 
                  train_dataset, 
                  'C4LSTM', 
                  criterion, 
                  k_folds = k_folds,
                  num_epochs = num_epoch, 
                  dropout = dropout,
                  print_every = print_every,
                  stack_num = 4,
                  subsampler = True, 
                  lr_decay=True, 
                  verbose = True,
                  cross_valid = False,
                  bidirection = True,
                  if_save = True)
acc_last_model= test(test_dataset, model_LSTM)
print("150 Epochs for LSTM need %s seconds ---" % (time.time() - start_time))

Cross validation is not implemented
===============FOLD 0===============
Model Initialized
Loss afer 25 epoch: 0.705, with the accuracy 73.422
The validation accuracy at epoch 25 is: 81.25
Loss afer 50 epoch: 0.436, with the accuracy 83.766
The validation accuracy at epoch 50 is: 90.62
Loss afer 75 epoch: 0.298, with the accuracy 89.047
The validation accuracy at epoch 75 is: 99.22
All 5 Folds Completed
Fold 0: 98.25697354917173 %
Average is:  98.25697354917173 %
Saving training results...
The test accuracy is: 68.12 %
150 Epochs for LSTM need 103.95883774757385 seconds ---


# 3.GRU

In [6]:
start_time = time.time()
num_epoch = 60
dropout = 0.5
cross_valid = False
model_GRU = train(x_shape, 
                  train_dataset, 
                  'C4GRU', 
                  criterion, 
                  k_folds = k_folds,
                  num_epochs = num_epoch, 
                  dropout = dropout,
                  print_every = print_every,
                  stack_num = 4,
                  subsampler = True, 
                  lr_decay=True, 
                  verbose = True,
                  cross_valid = False,
                  bidirection = False)
acc_last_model= test(test_dataset, model_GRU)
print("150 Epochs for GRU need %s seconds ---" % (time.time() - start_time))

Cross validation is not implemented
===============FOLD 0===============
Model Initialized
Loss afer 25 epoch: 0.729, with the accuracy 72.281
The validation accuracy at epoch 25 is: 82.03
Loss afer 50 epoch: 0.443, with the accuracy 83.484
The validation accuracy at epoch 50 is: 92.19
All 5 Folds Completed
Fold 0: 96.16548015453198 %
Average is:  96.16548015453198 %
Saving training results...
The test accuracy is: 68.23 %
150 Epochs for GRU need 76.66164994239807 seconds ---


# 4.Optimization

In [8]:
from collections import defaultdict
dropout_list = np.linspace(0.5,0.8,4) #4
num_epoch_list = np.linspace(50,150,11,dtype=int) #11
num_stack_list = np.linspace(2,5,4,dtype=int)# 6

CNN_acc_list = []
LSTM_acc_list = []
GRU_acc_list = []
print(dropout_list)

[0.5 0.6 0.7 0.8]


In [9]:
largest_acc = 0.0
best_dropout = 0.0
best_num = 0
cnt = 0
for dropout in dropout_list:
    for num_epoch in num_epoch_list:
        cnt += 1
        key = (dropout,num_epoch)
        model_CNN = train(x_shape, 
                  train_dataset, 
                  'CNN', 
                  criterion, 
                  k_folds = k_folds,
                  num_epochs = num_epoch, 
                  dropout = dropout,
                  print_every = print_every,
                  subsampler = True, 
                  lr_decay=True, 
                  verbose = False,
                  cross_valid = False,
                  bidirection = False,
                  if_save = False)
        acc= test(test_dataset, model_CNN)
        if largest_acc < acc:
            largest_acc = acc
            best_dropout = dropout
            best_num = num_epoch
        CNN_acc_list.append([key,acc.item()])
        print('At iteration: %2d, now Best Acc is: %.3f' %(cnt, largest_acc))
print('Best Acc is: %.3f with dropout: %.2f and num_epoch: epoch: %2d' %(largest_acc, best_dropout, best_num))

Fold 0: 93.44440855803313 %
The test accuracy is: 70.77 %
At iteration:  1, now Best Acc is: 70.768
Fold 0: 95.93380680790654 %
The test accuracy is: 71.73 %
At iteration:  2, now Best Acc is: 71.727
Fold 0: 97.67524267126012 %
The test accuracy is: 74.66 %
At iteration:  3, now Best Acc is: 74.661
Fold 0: 98.65494198269315 %
The test accuracy is: 70.20 %
At iteration:  4, now Best Acc is: 74.661
Fold 0: 98.62107481779876 %
The test accuracy is: 72.35 %
At iteration:  5, now Best Acc is: 74.661
Fold 0: 99.1814289799443 %
The test accuracy is: 71.90 %
At iteration:  6, now Best Acc is: 74.661
Fold 0: 98.09018650761357 %
The test accuracy is: 72.12 %
At iteration:  7, now Best Acc is: 74.661
Fold 0: 98.56791037100332 %
The test accuracy is: 72.35 %
At iteration:  8, now Best Acc is: 74.661
Fold 0: 99.50421227349176 %
The test accuracy is: 71.33 %
At iteration:  9, now Best Acc is: 74.661
Fold 0: 98.69577987105758 %
The test accuracy is: 71.39 %
At iteration: 10, now Best Acc is: 74.661
F

In [10]:
print(CNN_acc_list)

[[(0.5, 50), 70.76750183105469], [(0.5, 60), 71.72686767578125], [(0.5, 70), 74.66140747070312], [(0.5, 80), 70.20316314697266], [(0.5, 90), 72.34762573242188], [(0.5, 100), 71.89616394042969], [(0.5, 110), 72.12190246582031], [(0.5, 120), 72.34762573242188], [(0.5, 130), 71.33182525634766], [(0.5, 140), 71.38825988769531], [(0.5, 150), 72.46049499511719], [(0.6, 50), 68.79232788085938], [(0.6, 60), 70.09029388427734], [(0.6, 70), 71.72686767578125], [(0.6, 80), 70.20316314697266], [(0.6, 90), 71.83972930908203], [(0.6, 100), 71.83972930908203], [(0.6, 110), 69.69526672363281], [(0.6, 120), 69.41309356689453], [(0.6, 130), 72.12190246582031], [(0.6, 140), 72.51692962646484], [(0.6, 150), 72.2347640991211], [(0.7000000000000001, 50), 61.568851470947266], [(0.7000000000000001, 60), 65.18058776855469], [(0.7000000000000001, 70), 65.23702239990234], [(0.7000000000000001, 80), 68.11512756347656], [(0.7000000000000001, 90), 67.21218872070312], [(0.7000000000000001, 100), 71.27539825439453], 

In [11]:
largest_acc_LSTM = 0.0
cnt = 0
for num_stack in num_stack_list:
    for num_epoch in num_epoch_list:
        cnt += 1
        key = (num_stack,num_epoch)
        model_LSTM = train(x_shape, 
                  train_dataset, 
                  'LSTM', 
                  criterion, 
                  k_folds = k_folds,
                  num_epochs = num_epoch, 
                  dropout = best_dropout,
                  print_every = print_every,
                  stack_num = num_stack,
                  subsampler = True, 
                  lr_decay=True, 
                  verbose = False,
                  cross_valid = False,
                  bidirection = False,
                  if_save = False)
        acc= test(test_dataset, model_LSTM)
        if largest_acc_LSTM < acc:
            largest_acc_LSTM = acc
            best_stack = num_stack
            best_num = num_epoch
        LSTM_acc_list.append([key,acc.item()])
        print('At iteration: %2d, now Best Acc is: %.3f' %(cnt, largest_acc_LSTM))
print('Best Acc is: %.3f with stack number: %.2f and num_epoch: epoch: %2d' %(largest_acc_LSTM, num_stack, best_num))

Fold 0: 93.2421462447555 %
The test accuracy is: 66.08 %
At iteration:  1, now Best Acc is: 66.084
Fold 0: 97.79087405734592 %
The test accuracy is: 65.07 %
At iteration:  2, now Best Acc is: 66.084
Fold 0: 98.61341518825955 %
The test accuracy is: 65.97 %
At iteration:  3, now Best Acc is: 66.084
Fold 0: 98.05196380615234 %
The test accuracy is: 67.78 %
At iteration:  4, now Best Acc is: 67.777
Fold 0: 99.30338880750868 %
The test accuracy is: 64.56 %
At iteration:  5, now Best Acc is: 67.777
Fold 0: 99.47200605604384 %
The test accuracy is: 63.66 %
At iteration:  6, now Best Acc is: 67.777
Fold 0: 99.02036963568793 %
The test accuracy is: 66.31 %
At iteration:  7, now Best Acc is: 67.777
Fold 0: 99.21719106038411 %
The test accuracy is: 65.63 %
At iteration:  8, now Best Acc is: 67.777
Fold 0: 99.77560142234519 %
The test accuracy is: 64.33 %
At iteration:  9, now Best Acc is: 67.777
Fold 0: 99.46371714274089 %
The test accuracy is: 64.95 %
At iteration: 10, now Best Acc is: 67.777
F

In [12]:
print(LSTM_acc_list)

[[(2, 50), 66.0835189819336], [(2, 60), 65.06771850585938], [(2, 70), 65.97064971923828], [(2, 80), 67.77652740478516], [(2, 90), 64.55982208251953], [(2, 100), 63.656883239746094], [(2, 110), 66.30925750732422], [(2, 120), 65.6320571899414], [(2, 130), 64.33409118652344], [(2, 140), 64.95484924316406], [(2, 150), 67.26862335205078], [(3, 50), 66.0835189819336], [(3, 60), 67.09932708740234], [(3, 70), 66.47855377197266], [(3, 80), 68.79232788085938], [(3, 90), 65.01128387451172], [(3, 100), 68.34085845947266], [(3, 110), 65.80136108398438], [(3, 120), 65.68849182128906], [(3, 130), 66.70429229736328], [(3, 140), 67.09932708740234], [(3, 150), 67.21218872070312], [(4, 50), 64.50338745117188], [(4, 60), 65.34989166259766], [(4, 70), 65.8577880859375], [(4, 80), 69.92098999023438], [(4, 90), 66.422119140625], [(4, 100), 66.13995361328125], [(4, 110), 66.87358856201172], [(4, 120), 68.39729309082031], [(4, 130), 67.32505798339844], [(4, 140), 69.13092803955078], [(4, 150), 67.6072235107421

In [13]:
largest_acc_GRU = 0.0
cnt = 0
for num_stack in num_stack_list:
    for num_epoch in num_epoch_list:
        cnt += 1
        key = (num_stack,num_epoch)
        model_GRU = train(x_shape, 
                  train_dataset, 
                  'GRU', 
                  criterion, 
                  k_folds = k_folds,
                  num_epochs = num_epoch, 
                  dropout = best_dropout,
                  print_every = print_every,
                  stack_num = num_stack,
                  subsampler = True, 
                  lr_decay=True, 
                  verbose = False,
                  cross_valid = False,
                  bidirection = True,
                  if_save = False)
        acc= test(test_dataset, model_GRU)
        if largest_acc_GRU < acc:
            largest_acc_GRU = acc
            best_stack = num_stack
            best_num = num_epoch
        GRU_acc_list.append([key,acc.item()])
        print('At iteration: %2d, now Best Acc is: %.3f' %(cnt, largest_acc_GRU))
print('Best Acc is: %.3f with stack number: %.2f and num_epoch: epoch: %2d' %(largest_acc_GRU, num_stack, best_num))

Fold 0: 96.6848240605107 %
The test accuracy is: 65.80 %
At iteration:  1, now Best Acc is: 65.801
Fold 0: 96.72838959870515 %
The test accuracy is: 66.65 %
At iteration:  2, now Best Acc is: 66.648
Fold 0: 99.04674219202113 %
The test accuracy is: 66.48 %
At iteration:  3, now Best Acc is: 66.648
Fold 0: 99.30813485604746 %
The test accuracy is: 67.66 %
At iteration:  4, now Best Acc is: 67.664
Fold 0: 99.42058648003473 %
The test accuracy is: 64.22 %
At iteration:  5, now Best Acc is: 67.664
Fold 0: 99.90989939371745 %
The test accuracy is: 65.80 %
At iteration:  6, now Best Acc is: 67.664
Fold 0: 99.65706521493418 %
The test accuracy is: 68.28 %
At iteration:  7, now Best Acc is: 68.284
Fold 0: 98.76731476960359 %
The test accuracy is: 66.76 %
At iteration:  8, now Best Acc is: 68.284
Fold 0: 99.213458025897 %
The test accuracy is: 66.70 %
At iteration:  9, now Best Acc is: 68.284
Fold 0: 99.03294033474393 %
The test accuracy is: 69.86 %
At iteration: 10, now Best Acc is: 69.865
Fol

In [14]:
print(GRU_acc_list)

[[(2, 50), 65.80136108398438], [(2, 60), 66.64785766601562], [(2, 70), 66.47855377197266], [(2, 80), 67.66365814208984], [(2, 90), 64.22122192382812], [(2, 100), 65.80136108398438], [(2, 110), 68.28443145751953], [(2, 120), 66.76072692871094], [(2, 130), 66.70429229736328], [(2, 140), 69.86456298828125], [(2, 150), 66.76072692871094], [(3, 50), 64.84198760986328], [(3, 60), 66.36569213867188], [(3, 70), 67.15576171875], [(3, 80), 68.34085845947266], [(3, 90), 67.7200927734375], [(3, 100), 63.431156158447266], [(3, 110), 67.04289245605469], [(3, 120), 68.28443145751953], [(3, 130), 67.88938903808594], [(3, 140), 70.25959777832031], [(3, 150), 66.47855377197266], [(4, 50), 65.57562255859375], [(4, 60), 70.71106719970703], [(4, 70), 66.30925750732422], [(4, 80), 67.43791961669922], [(4, 90), 67.15576171875], [(4, 100), 65.68849182128906], [(4, 110), 66.1963882446289], [(4, 120), 65.23702239990234], [(4, 130), 66.70429229736328], [(4, 140), 65.5191879272461], [(4, 150), 67.7200927734375], 

# Optimize 2 - Try Pure RNN and C2RNN

In [11]:
start_time = time.time()
num_epoch = 10
dropout = 0.5
cross_valid = False
num_stack = 4
model_LSTM = train(x_shape, 
                  train_dataset, 
                  'LSTM', 
                  criterion, 
                  k_folds = k_folds,
                  num_epochs = num_epoch, 
                  dropout = dropout,
                  print_every = 10,
                  stack_num = num_stack,
                  subsampler = False, 
                  lr_decay=True, 
                  verbose = True,
                  cross_valid = False,
                  bidirection = True,
                  if_save = False)
acc_last_model= test(test_dataset, model_LSTM)
print("10 Epochs for Pure LSTM need %s seconds ---" % (time.time() - start_time))

Cross validation is not implemented
===============FOLD 0===============
Model Initialized
Loss afer 10 epoch: 0.043, with the average accuracy 98.767
The validation accuracy at epoch 10 is: 97.66
All 5 Folds Completed
Fold 0: 98.00751664903429 %
Average is:  98.00751664903429 %
Saving training results...
The test accuracy is: 64.33 %
10 Epochs for Pure LSTM need 89.56885385513306 seconds ---


In [12]:
num_epoch = 10
model_LSTM = train(x_shape, 
                  train_dataset, 
                  'GRU', 
                  criterion, 
                  k_folds = k_folds,
                  num_epochs = num_epoch, 
                  dropout = dropout,
                  print_every = 10,
                  stack_num = num_stack,
                  subsampler = False, 
                  lr_decay=True, 
                  verbose = True,
                  cross_valid = False,
                  bidirection = True,
                  if_save = False)
acc_last_model= test(test_dataset, model_LSTM)
print("10 Epochs for Pure GRU need %s seconds ---" % (time.time() - start_time))

Cross validation is not implemented
===============FOLD 0===============
Model Initialized
Loss afer 10 epoch: 0.053, with the average accuracy 99.025
The validation accuracy at epoch 10 is: 94.53
All 5 Folds Completed
Fold 0: 97.51563347710504 %
Average is:  97.51563347710504 %
Saving training results...
The test accuracy is: 65.52 %
10 Epochs for Pure GRU need 170.93080520629883 seconds ---


In [13]:
num_epoch = 100
dropout = 0.5
cross_valid = False
num_stack = 4
model_CNN = train(x_shape2, 
                  train_dataset2, 
                  'CNN', 
                  criterion, 
                  k_folds = k_folds,
                  num_epochs = num_epoch, 
                  dropout = dropout,
                  print_every = 25,
                  stack_num = num_stack,
                  subsampler = False, 
                  lr_decay=True, 
                  verbose = True,
                  cross_valid = False,
                  bidirection = True,
                  if_save = False)
acc_last_model= test(test_dataset2, model_CNN)

Cross validation is not implemented
===============FOLD 0===============
Model Initialized
Loss afer 25 epoch: 0.999, with the average accuracy 56.521
The validation accuracy at epoch 25 is: 32.03
Loss afer 50 epoch: 0.979, with the average accuracy 59.226
The validation accuracy at epoch 50 is: 33.59
Loss afer 75 epoch: 0.970, with the average accuracy 61.457
The validation accuracy at epoch 75 is: 32.03
Loss afer 100 epoch: 0.970, with the average accuracy 56.492
The validation accuracy at epoch 100 is: 33.59
All 5 Folds Completed
Fold 0: 37.555619376046316 %
Average is:  37.555619376046316 %
Saving training results...
The test accuracy is: 42.44 %


In [6]:
num_epoch = 60
dropout = 0.5
cross_valid = False
num_stack = 4
model_C4GRU = train(x_shape2, 
                  train_dataset2, 
                  'C4GRU', 
                  criterion, 
                  k_folds = k_folds,
                  num_epochs = num_epoch, 
                  dropout = dropout,
                  print_every = 25,
                  stack_num = num_stack,
                  subsampler = False, 
                  lr_decay=True, 
                  verbose = True,
                  cross_valid = False,
                  bidirection = True,
                  if_save = False)
acc_last_model= test(test_dataset2, model_C4GRU)

Cross validation is not implemented
===============FOLD 0===============


C:\Users\Rong\anaconda3\lib\site-packages\torch\nn\modules\conv.py:442: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  ..\aten\src\ATen\native\Convolution.cpp:647.)
  return F.conv2d(input, weight, bias, self.stride,


Model Initialized
Loss afer 25 epoch: 1.055, with the average accuracy 52.919
The validation accuracy at epoch 25 is: 43.75
Loss afer 50 epoch: 0.809, with the average accuracy 66.245
The validation accuracy at epoch 50 is: 52.34
All 5 Folds Completed
Fold 0: 57.5751702444894 %
Average is:  57.5751702444894 %
Saving training results...
The test accuracy is: 56.21 %


In [7]:
num_epoch = 80
dropout = 0.5
cross_valid = False
num_stack = 4
model_C4LSTM = train(x_shape2, 
                  train_dataset2, 
                  'C4LSTM', 
                  criterion, 
                  k_folds = k_folds,
                  num_epochs = num_epoch, 
                  dropout = dropout,
                  print_every = 25,
                  stack_num = num_stack,
                  subsampler = False, 
                  lr_decay=True, 
                  verbose = True,
                  cross_valid = False,
                  bidirection = True,
                  if_save = False)
acc_last_model= test(test_dataset2, model_C4LSTM)

Cross validation is not implemented
===============FOLD 0===============
Model Initialized
Loss afer 25 epoch: 1.182, with the average accuracy 45.245
The validation accuracy at epoch 25 is: 30.47
Loss afer 50 epoch: 1.186, with the average accuracy 47.363
The validation accuracy at epoch 50 is: 29.69
Loss afer 75 epoch: 1.177, with the average accuracy 45.289
The validation accuracy at epoch 75 is: 28.91
All 5 Folds Completed
Fold 0: 28.680424009050643 %
Average is:  28.680424009050643 %
Saving training results...
The test accuracy is: 30.02 %


In [14]:
num_epoch = 10
dropout = 0.5
cross_valid = False
saving_file = False
model_LSTM = train(x_shape2, 
                  train_dataset2, 
                  'LSTM', 
                  criterion, 
                  k_folds = k_folds,
                  num_epochs = num_epoch,
                  stack_num = 4,
                  dropout = dropout,
                  print_every = 25,
                  subsampler = False, 
                  lr_decay=True, 
                  verbose = True,
                  cross_valid = False,
                  bidirection = True,
                  if_save = False)
acc_last_model= test(test_dataset2, model_LSTM)

Cross validation is not implemented
===============FOLD 0===============
Model Initialized
All 5 Folds Completed
Fold 0: 52.43572834559849 %
Average is:  52.43572834559849 %
Saving training results...
The test accuracy is: 57.34 %


In [15]:
num_epoch = 10
dropout = 0.5
cross_valid = False
saving_file = False
model_GRU = train(x_shape2, 
                  train_dataset2, 
                  'GRU', 
                  criterion, 
                  k_folds = k_folds,
                  num_epochs = num_epoch,
                  stack_num = 4,
                  dropout = dropout,
                  print_every = 25,
                  subsampler = False, 
                  lr_decay=True, 
                  verbose = True,
                  cross_valid = False,
                  bidirection = True,
                  if_save = False)
acc_last_model= test(test_dataset2, model_GRU)

Cross validation is not implemented
===============FOLD 0===============
Model Initialized
All 5 Folds Completed
Fold 0: 58.09315490722656 %
Average is:  58.09315490722656 %
Saving training results...
The test accuracy is: 56.66 %
